In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import from_levels_and_colors
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import xarray as xr
from netCDF4 import Dataset
from wrf import (to_np, getvar, smooth2d, get_cartopy, cartopy_xlim,cartopy_ylim, latlon_coords)

In [5]:
filename = "wrfout_d03_2012-07-24_00:00:00"

In [8]:
# specify the location of the data
path_in = "data_for_practice/"
path_out = "data/"
ds= xr.open_dataset(path_in + 'wrfout_d03_2012-07-24_00:00:00')

In [9]:
print(ds)

<xarray.Dataset>
Dimensions:                (Time: 1, bottom_top: 49, bottom_top_stag: 50, seed_dim_stag: 33, soil_layers_stag: 4, south_north: 627, south_north_stag: 628, west_east: 642, west_east_stag: 643)
Coordinates:
    XLAT                   (Time, south_north, west_east) float32 ...
    XLONG                  (Time, south_north, west_east) float32 ...
    XTIME                  (Time) datetime64[ns] ...
    XLAT_U                 (Time, south_north, west_east_stag) float32 ...
    XLONG_U                (Time, south_north, west_east_stag) float32 ...
    XLAT_V                 (Time, south_north_stag, west_east) float32 ...
    XLONG_V                (Time, south_north_stag, west_east) float32 ...
Dimensions without coordinates: Time, bottom_top, bottom_top_stag, seed_dim_stag, soil_layers_stag, south_north, south_north_stag, west_east, west_east_stag
Data variables:
    Times                  (Time) |S19 ...
    LU_INDEX               (Time, south_north, west_east) float32 ...

In [61]:
u=ds.U
u

<xarray.DataArray 'U' (Time: 1, bottom_top: 49, south_north: 627, west_east_stag: 643)>
[19754889 values with dtype=float32]
Coordinates:
    XTIME    (Time) datetime64[ns] 2012-07-24
    XLAT_U   (Time, south_north, west_east_stag) float32 ...
    XLONG_U  (Time, south_north, west_east_stag) float32 ...
Dimensions without coordinates: Time, bottom_top, south_north, west_east_stag
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  x-wind component
    units:        m s-1
    stagger:      X

In [36]:
u.shape

(1, 49, 627, 643)

In [37]:
u1=u[0,:]
#just select the first time index, it will create a dataarray with no time dimension
u1.shape
u1

<xarray.DataArray 'U' (bottom_top: 49, south_north: 627, west_east_stag: 643)>
[19754889 values with dtype=float32]
Coordinates:
    XTIME    datetime64[ns] 2012-07-24
    XLAT_U   (south_north, west_east_stag) float32 ...
    XLONG_U  (south_north, west_east_stag) float32 ...
Dimensions without coordinates: bottom_top, south_north, west_east_stag
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  x-wind component
    units:        m s-1
    stagger:      X

In [38]:
u2= u.isel(Time=0)
u2.shape
print(u2)
print(u)

<xarray.DataArray 'U' (bottom_top: 49, south_north: 627, west_east_stag: 643)>
[19754889 values with dtype=float32]
Coordinates:
    XTIME    datetime64[ns] 2012-07-24
    XLAT_U   (south_north, west_east_stag) float32 ...
    XLONG_U  (south_north, west_east_stag) float32 ...
Dimensions without coordinates: bottom_top, south_north, west_east_stag
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  x-wind component
    units:        m s-1
    stagger:      X
<xarray.DataArray 'U' (Time: 1, bottom_top: 49, south_north: 627, west_east_stag: 643)>
[19754889 values with dtype=float32]
Coordinates:
    XTIME    (Time) datetime64[ns] 2012-07-24
    XLAT_U   (Time, south_north, west_east_stag) float32 ...
    XLONG_U  (Time, south_north, west_east_stag) float32 ...
Dimensions without coordinates: Time, bottom_top, south_north, west_east_stag
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  x-wind component
    units:        m s-1
    stagger:    

In [39]:
u_sel_HK=u.sel(south_north= slice(20,24), west_east_stag=slice(112,116))
print(u_sel_HK)


<xarray.DataArray 'U' (Time: 1, bottom_top: 49, south_north: 4, west_east_stag: 4)>
array([[[[ 13.071158, ...,  13.421521],
         ...,
         [ 13.679166, ...,  13.742548]],

        ...,

        [[-19.180454, ..., -19.49115 ],
         ...,
         [-19.171036, ..., -19.465782]]]], dtype=float32)
Coordinates:
    XTIME    (Time) datetime64[ns] 2012-07-24
    XLAT_U   (Time, south_north, west_east_stag) float32 ...
    XLONG_U  (Time, south_north, west_east_stag) float32 ...
Dimensions without coordinates: Time, bottom_top, south_north, west_east_stag
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  x-wind component
    units:        m s-1
    stagger:      X


In [40]:
u_sel_HK_isel_time=u.sel(south_north= slice(20,24), west_east_stag=slice(112,116)).isel(Time=0)
print(u_sel_HK_isel_time)

<xarray.DataArray 'U' (bottom_top: 49, south_north: 4, west_east_stag: 4)>
array([[[ 13.071158,  12.979356,  13.108482,  13.421521],
        [ 13.049014,  12.998456,  13.154517,  13.448538],
        [ 13.218992,  13.181503,  13.29812 ,  13.534159],
        [ 13.679166,  13.653292,  13.663485,  13.742548]],

       [[ 14.309056,  14.15559 ,  14.203631,  14.471278],
        [ 14.287229,  14.158194,  14.241831,  14.497403],
        [ 14.474984,  14.353732,  14.409131,  14.62409 ],
        [ 15.004869,  14.934891,  14.89845 ,  14.950585]],

       ...,

       [[-19.779585, -19.862482, -19.937767, -20.000137],
        [-19.781532, -19.866793, -19.944313, -20.012573],
        [-19.781048, -19.86524 , -19.943336, -20.012943],
        [-19.781311, -19.862919, -19.939665, -20.007275]],

       [[-19.180454, -19.269875, -19.374096, -19.49115 ],
        [-19.178904, -19.26937 , -19.37199 , -19.486652],
        [-19.174229, -19.264212, -19.365473, -19.477407],
        [-19.171036, -19.259857, -19

In [59]:
#u_sel_nearest_location= u.sel(XLAT_U=22, XLONG_V=114, method='nearest')
u_sel_nearest_location=u.sel(south_north=22, west_east_stag=112)
print(u_sel_nearest_location)

<xarray.DataArray 'U' (Time: 1, bottom_top: 49)>
array([[ 13.218992,  14.474984,  15.33024 ,  16.402603,  17.448126,
         18.483759,  19.70778 ,  21.032475,  22.396942,  22.723896,
         21.496643,  20.037682,  18.5756  ,  17.201788,  16.68617 ,
         16.996891,  17.017704,  17.162418,  17.01669 ,  16.374222,
         19.339811,  20.295794,  16.208666,  11.84599 ,   8.201128,
          8.453808,   7.184738,   6.29252 ,   6.471925,   5.865516,
          4.31229 ,   0.421587,  -3.434668,  -5.701296,  -9.181315,
        -11.701923, -14.86885 , -16.511475, -18.60773 , -20.425253,
        -24.920834, -28.987518, -22.15154 , -18.461412, -16.809383,
        -17.046425, -18.739769, -19.781048, -19.174229]], dtype=float32)
Coordinates:
    XTIME    (Time) datetime64[ns] 2012-07-24
    XLAT_U   (Time) float32 ...
    XLONG_U  (Time) float32 ...
Dimensions without coordinates: Time, bottom_top
Attributes:
    FieldType:    104
    MemoryOrder:  XYZ
    description:  x-wind component
   